In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [6]:
# Load data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
valid_df = pd.read_csv('validation.csv')

# Create label mapping
label_mapping = {
    'ChatGPT': 0,
    'Reddit': 1,
    'Wikipedia': 2,
    'LinkedIn': 3
}

# Convert label column using mapping
train_df.iloc[:, -1] = train_df.iloc[:, -1].map(label_mapping)
test_df.iloc[:, -1] = test_df.iloc[:, -1].map(label_mapping)
valid_df.iloc[:, -1] = valid_df.iloc[:, -1].map(label_mapping)

# Check for any NaN values in labels 
print(f"NaN values in train labels: {train_df.iloc[:, -1].isna().sum()}")
print(f"NaN values in test labels: {test_df.iloc[:, -1].isna().sum()}")
print(f"NaN values in validation labels: {valid_df.iloc[:, -1].isna().sum()}")


# Convert feature columns to numeric and handle NaN values
for col in train_df.columns[:-1]:  # exclude label column
    train_df[col] = pd.to_numeric(train_df[col], errors='coerce')
    test_df[col] = pd.to_numeric(test_df[col], errors='coerce')
    valid_df[col] = pd.to_numeric(valid_df[col], errors='coerce')

# Fill NaN values with column means instead of dropping rows
train_df = train_df.fillna(train_df.mean())
test_df = test_df.fillna(test_df.mean())
valid_df = valid_df.fillna(valid_df.mean())

# Verify data types before conversion to tensors
print(f"Data types in train_df:\n{train_df.dtypes}")

# Force convert to int64
train_df.iloc[:, -1] = train_df.iloc[:, -1].astype('float64')
test_df.iloc[:, -1] = test_df.iloc[:, -1].astype('float64')
valid_df.iloc[:, -1] = valid_df.iloc[:, -1].astype('float64')

NaN values in train labels: 0
NaN values in test labels: 0
NaN values in validation labels: 0
Data types in train_df:
Packet Count                    int64
Pkts_per_sec                  float64
Total Length                    int64
Bytes_per_sec                 float64
Norm Avg Interval             float64
Norm Max Interval             float64
Norm Min Interval             float64
Avg Length (bytes)            float64
Max Length (bytes)              int64
Min Length (bytes)              int64
Most Common Length (bytes)      int64
Label                           int64
dtype: object


In [9]:
X_train = train_df.drop(columns=['Label'])   
y_train = train_df['Label']                  

X_test = test_df.drop(columns=['Label'])    
y_test = test_df['Label']

rf = RandomForestClassifier(
    n_estimators=100,   
    max_depth=20,       
    random_state=42    
)

rf.fit(X_train, y_train)


y_pred = rf.predict(X_test)


acc = accuracy_score(y_test, y_pred)
print(f"Random Forest Test Accuracy: {acc*100:.2f}%")

f1 = f1_score(y_test, y_pred, average='macro')
print(f"Random Forest Test F1 Score: {f1*100:.2f}%")

Random Forest Test Accuracy: 91.67%
Random Forest Test F1 Score: 92.71%


In [10]:
from sklearn.metrics import accuracy_score, f1_score

X_train = train_df.drop(columns=['Label'])   
y_train = train_df['Label']

X_val = valid_df.drop(columns=['Label'])    
y_val = valid_df['Label']

rf = RandomForestClassifier(
    n_estimators=100,   
    max_depth=20,       
    random_state=42
)

rf.fit(X_train, y_train)

# Validate on the validation set
y_val_pred = rf.predict(X_val)

val_acc = accuracy_score(y_val, y_val_pred)
print(f"Random Forest Validation Accuracy: {val_acc*100:.2f}%")

val_f1 = f1_score(y_val, y_val_pred, average='macro')
print(f"Random Forest Validation F1 Score: {val_f1*100:.2f}%")



Random Forest Validation Accuracy: 95.83%
Random Forest Validation F1 Score: 95.80%


In [11]:
# Test Accuracy and F1 Visualization
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np

# --- Style Definitions (NYT Inspired - Reuse from previous) ---
# ... (style definitions remain the same) ...
colors = {
    'bar_lin': 'rgba(70, 130, 180, 0.8)',
    'bar_log': 'rgba(210, 105, 30, 0.8)',
    'grid': '#d9d9d9',
    'text': '#333333',
    'background': '#ffffff',
    'axis_line': '#cccccc',
    'heatmap': 'Blues' # Or 'Greys', 'YlGnBu', etc.
}
font_family = "Arial, sans-serif"
base_layout = go.Layout( # Shared layout elements
     # ... (base_layout definition remains the same) ...
    font=dict(family=font_family, size=12, color=colors['text']),
    title_font_size=18,
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    yaxis=dict(showgrid=True, gridcolor=colors['grid'], gridwidth=1, showline=False, ticks='outside', tickcolor=colors['axis_line'], zeroline=False),
    xaxis=dict(showgrid=False, showline=True, linecolor=colors['axis_line'], linewidth=1, ticks='outside', tickcolor=colors['axis_line'], zeroline=False),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1, bgcolor='rgba(0,0,0,0)', bordercolor=colors['background']),
    margin=dict(l=80, r=40, t=80, b=60)
)


# Basic metrics
rf_accuracy = accuracy_score(y_test, y_pred)
rf_f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=0)

# Classification report
website_names = sorted(y_test.unique())  # Ensure consistent label order
report_rf_dict = classification_report(y_val, y_val_pred, target_names=website_names, output_dict=True, zero_division=0)

models = ['Random Forest']
accuracies = [rf_accuracy]
f1_macros = [rf_f1_macro]

fig_metrics = go.Figure(layout=base_layout)

fig_metrics.add_trace(go.Bar(
    x=models,
    y=accuracies,
    name='Accuracy',
    marker_color=colors['bar_lin'],
    text=[f'{acc:.3f}' for acc in accuracies],
    textposition='auto',
    width=0.3
))

fig_metrics.add_trace(go.Bar(
    x=models,
    y=f1_macros,
    name='Macro F1-Score',
    marker_color=colors['bar_log'],
    text=[f'{f1:.3f}' for f1 in f1_macros],
    textposition='auto',
    width=0.3
))

fig_metrics.update_layout(
    title_text='Random Forest Performance',
    xaxis_title='Model',
    yaxis_title='Score',
    yaxis_range=[0, 1.05],
    yaxis_tickformat=".2f",
    barmode='group',
    legend_title_text='Metric'
)

fig_metrics.show()

In [12]:
# Valid Accuracy and F1 Visualization
# cm = confusion_matrix(y_test, y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_mapping.keys())
# disp.plot()
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np

# --- Style Definitions (NYT Inspired - Reuse from previous) ---
# ... (style definitions remain the same) ...
colors = {
    'bar_lin': 'rgba(70, 130, 180, 0.8)',
    'bar_log': 'rgba(210, 105, 30, 0.8)',
    'grid': '#d9d9d9',
    'text': '#333333',
    'background': '#ffffff',
    'axis_line': '#cccccc',
    'heatmap': 'Blues' # Or 'Greys', 'YlGnBu', etc.
}
font_family = "Arial, sans-serif"
base_layout = go.Layout( # Shared layout elements
     # ... (base_layout definition remains the same) ...
    font=dict(family=font_family, size=12, color=colors['text']),
    title_font_size=18,
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    yaxis=dict(showgrid=True, gridcolor=colors['grid'], gridwidth=1, showline=False, ticks='outside', tickcolor=colors['axis_line'], zeroline=False),
    xaxis=dict(showgrid=False, showline=True, linecolor=colors['axis_line'], linewidth=1, ticks='outside', tickcolor=colors['axis_line'], zeroline=False),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1, bgcolor='rgba(0,0,0,0)', bordercolor=colors['background']),
    margin=dict(l=80, r=40, t=80, b=60)
)


# Basic metrics
rf_accuracy = accuracy_score(y_val, y_val_pred)
rf_f1_macro = f1_score(y_val, y_val_pred, average='macro', zero_division=0)

# Classification report
website_names = sorted(y_test.unique())  # Ensure consistent label order
report_rf_dict = classification_report(y_test, y_pred, target_names=website_names, output_dict=True, zero_division=0)

models = ['Random Forest']
accuracies = [rf_accuracy]
f1_macros = [rf_f1_macro]

fig_metrics = go.Figure(layout=base_layout)

fig_metrics.add_trace(go.Bar(
    x=models,
    y=accuracies,
    name='Accuracy',
    marker_color=colors['bar_lin'],
    text=[f'{acc:.3f}' for acc in accuracies],
    textposition='auto',
    width=0.3
))

fig_metrics.add_trace(go.Bar(
    x=models,
    y=f1_macros,
    name='Macro F1-Score',
    marker_color=colors['bar_log'],
    text=[f'{f1:.3f}' for f1 in f1_macros],
    textposition='auto',
    width=0.3
))

fig_metrics.update_layout(
    title_text='Random Forest Performance',
    xaxis_title='Model',
    yaxis_title='Score',
    yaxis_range=[0, 1.05],
    yaxis_tickformat=".2f",
    barmode='group',
    legend_title_text='Metric'
)

fig_metrics.show()


In [13]:
# Test Confusion Matrix
import numpy as np
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# === Step 1: Define your label mapping (string → int)
label_mapping = {
    'ChatGPT': 0,
    'Reddit': 1,
    'Wikipedia': 2,
    'LinkedIn': 3
}

# === Step 2: Invert mapping to get int → string (for display labels)
index_to_label = {v: k for k, v in label_mapping.items()}

# === Step 3: Use numeric labels for confusion matrix
numeric_labels = list(index_to_label.keys())  # [0, 1, 2, 3]
cm = confusion_matrix(y_test, y_pred, labels=numeric_labels)

# === Step 4: Get the website names in correct numeric order
website_names = [index_to_label[i] for i in numeric_labels]

# === Step 5: Create the heatmap
def create_confusion_matrix_heatmap(cm, labels, title, color_scale='Blues'):
    text_values = [[str(val) for val in row] for row in cm]

    fig = ff.create_annotated_heatmap(
        z=cm,
        x=labels,
        y=labels,
        annotation_text=text_values,
        colorscale=color_scale,
        showscale=True
    )

    fig.update_layout(
        title_text=title,
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
        xaxis_side="bottom",
        yaxis_autorange='reversed',
        xaxis=dict(tickangle=-45),
        margin=dict(l=100, r=50, t=100, b=100),
        font=dict(family='Arial', size=12, color='black'),
        plot_bgcolor='white',
        paper_bgcolor='white'
    )

    fig.update_xaxes(type='category')
    fig.update_yaxes(type='category')
    return fig

# === Step 6: Plot the figure
fig_cm = create_confusion_matrix_heatmap(
    cm, website_names, 'Confusion Matrix: Random Forest', color_scale='Blues'
)

fig_cm.show()





In [14]:
import plotly.graph_objects as go
import numpy as np

# Ensure the following variables are defined from your notebook execution:
# y_test: The true numeric labels for the test set
# y_pred_lin: The predicted numeric labels from Linear Regression (rounded)
# y_pred_log: The predicted numeric labels from Logistic Regression
# label_encoder: The fitted LabelEncoder instance
# website_names: The array of class names (e.g., ['ChatGPT', 'LinkedIn', ...])

# --- Create Plotly Plot for Linear Regression ---

fig_lin_plotly = go.Figure()

# Add prediction trace
fig_lin_plotly.add_trace(go.Scatter(
    y=y_pred,
    mode='lines+markers',
    name='prediction',
    line=dict(color='navy', width=2),
    marker=dict(size=5)
))

# Add real values trace
fig_lin_plotly.add_trace(go.Scatter(
    y=y_test,
    mode='lines+markers',
    name='real_values',
    line=dict(color='lightcoral', width=2, dash='dash'), # Use dash for differentiation
    marker=dict(size=5),
    opacity=0.8
))

# Update layout
fig_lin_plotly.update_layout(
    title='Random Forests Classification',
    xaxis_title='Sample Index',
    yaxis_title='Website Label',
    yaxis=dict(
        tickmode='array',
        tickvals=np.arange(len(website_names)), # Numeric positions (0, 1, 2, 3)
        ticktext=website_names # Text labels ('ChatGPT', 'LinkedIn', ...)
    ),
    legend_title_text='Legend',
    template='plotly_white' # Or another template like 'plotly_dark', 'ggplot2'
)

fig_lin_plotly.show()


